In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from astropy.modeling.models import custom_model
from astropy.modeling import Fittable1DModel, Parameter
from astroquery.simbad import Simbad
import juliandate as jd
from datetime import datetime, timedelta
import pandas as pd
from time import sleep

We have N stars each with period p_(n), p_(n+1), etc... We need to maximise phase coverage for our observations, i.e. no observing that star at the same phase more than once.

How to do this: For a specific star, n, calculate time difference between previous observations and the time for the current observation, del_t = t_0 - t_i. Divide these times by the period, phase_t = del_t/p. Discard all phase_t except for the one closest to a whole integer. If phase_t is an integer then we are observing at a phase already observed. For all N stars find the phase_t furthest from an integer (closest to 0.5, 1.5, ...) and observe it. Repeat untill we have measurements for the entire phase for all N stars.

Code:

current_time = getDate
star_greatest_offset = 0
target_star = []

for star in starsCatalog:
    greatest_offset = 0
    for observation in star["observations"]:
        phase_t = []
        offset = (current_time - observation["date"] / star["period"]) % 1
        if offset >= 0.5:
            offset = 1 - offset
        if offset > greatest_offset:
            greatest_offset = offset
    if greatest_offset > star_greatest_offset:
        target_star = star

//observe target_star


In [2]:
ts = pd.Timestamp(datetime.now())
current_time = ts.to_julian_date()
print(ts)
print(current_time)

2025-10-07 13:22:08.707503
2460956.0570452255


In [3]:
# Create star catalogue
target_stars = ('RV UMa', 'RZ Cep', 'RR Lyr')

#for star in target_stars:
query = f"""
SELECT basic.main_id, vartyp AS type, ROUND(AVG(period), 6) AS period
FROM basic
JOIN mesVar ON basic.oid = mesVar.oidref
WHERE basic.otype = 'RR*'
AND main_id IN {target_stars}
GROUP BY basic.main_id, vartyp
HAVING vartyp = 'RRAB' OR vartyp = 'RRC'
"""
result = Simbad.query_tap(query)
print(result)
stars_df = result.to_pandas()
stars_df.main_id = [ star.replace('V*', '').strip() for star in stars_df.main_id]
stars_df

 main_id  type  period 
--------- ---- --------
V* RR Lyr RRAB 0.566787
V* RV UMa RRAB 0.468071
V* RZ Cep  RRC 0.308706


,main_id,type,period
0,RR Lyr,RRAB,0.566787
1,RV UMa,RRAB,0.468071
2,RZ Cep,RRC,0.308706


In [4]:
#Create dummy observation times

dummy_time = pd.Timestamp('2025-09-23 21:09:02.442422')
dummy_obs = []
for i in range(15):
    dummy_julian_time = dummy_time.to_julian_date()
    dummy_time = dummy_time + timedelta(minutes = 10)
    dummy_obs = np.append(dummy_obs, dummy_julian_time)

#print(dummy_obs)

RV_UMa_dummy_obs = dummy_obs[0::3]
RZ_Cep_dummy_obs = dummy_obs[1::3]
RR_Lyr_dummy_obs = dummy_obs[2::3]

#print(RV_UMa_dummy_obs)
#print(RZ_Cep_dummy_obs)
#print(RR_Lyr_dummy_obs)


In [5]:
observations_df = pd.DataFrame(
    {
        "main_id": target_stars,
        "obs_times": [RV_UMa_dummy_obs, RZ_Cep_dummy_obs, RR_Lyr_dummy_obs]
    }
)

observations_df

,main_id,obs_times
0,RV UMa,"[2460942.381278269, 2460942.402111602, 2460942..."
1,RZ Cep,"[2460942.388222713, 2460942.4090560465, 246094..."
2,RR Lyr,"[2460942.3951671575, 2460942.416000491, 246094..."


In [6]:
catalogue_df = pd.merge(stars_df, observations_df, on="main_id")
catalogue_df

,main_id,type,period,obs_times
0,RR Lyr,RRAB,0.566787,"[2460942.3951671575, 2460942.416000491, 246094..."
1,RV UMa,RRAB,0.468071,"[2460942.381278269, 2460942.402111602, 2460942..."
2,RZ Cep,RRC,0.308706,"[2460942.388222713, 2460942.4090560465, 246094..."


In [9]:
star_greatest_offset = 0
target_star = ""

for index, row in catalogue_df.iterrows():
    greatest_offset = 0
    for obs_time in row["obs_times"]:
        offset = (current_time - obs_time/ row["period"]) % 1
        if offset >= 0.5:
            offset = 1 - offset
        if offset > greatest_offset:
            greatest_offset = offset
    if greatest_offset > star_greatest_offset:
        star_greatest_offset = greatest_offset
        target_star = row["main_id"]
        print(star_greatest_offset)
        
print(target_star)

0.4816430131904781
RR Lyr


In [8]:
#for index, row in catalogue_df.iterrows():
#    greatest_offset = 0
#    for obs_times in row["obs_times"]:
#        phase_t = []
#        offset = (current_time - obs_time / row["period"]) % 1
#        if offset >= 0.5:
#            offset = 1 - offset
#        if offset > greatest_offset:
#            greatest_offset = offset
#    if greatest_offset > star_greatest_offset:
#        target_star = star